In [1]:
import numpy as np
import pandas as pd
import datetime
from sqlalchemy import create_engine
import plotly.express as px
from database_utils import add_new_unl, add_new_dot, clean_unleashed, clean_dot, get_level_2
import streamlit as st

In [2]:
market_segment_dict = {
    'Vending': 'rgb(56,149,73)',
    'Grocery': 'rgb(248,184,230)',
    'Alternate Retail': 'rgb(46,70,166)',
    'Canada': 'rgb(204,208,221)',
    'Online': 'rgb(106,87,63)',
    'Other': 'rgb(200,237,233)',
    'Convenience': 'rgb(233,81,46)',
    'Broadline Distributor': 'rgb(233,152,19)',
    'Samples': 'rgb(141,62,92)'
    }

In [3]:
# database connection

db_password = "UnitCircle42!"
db_user = "postgres"
db_name = "dot"
endpoint = "awakedb.cre3f7yk1unp.us-west-1.rds.amazonaws.com"

connection_string = f"postgresql://{db_user}:{db_password}@{endpoint}:5432/{db_name}"
engine = create_engine(connection_string)

In [ ]:
# UPLOAD AND CLEAN DIRECT

unl_download = r"C:\Users\mikej\Downloads\SalesEnquiryList - 2024-03-30T063201.166.xlsx" ### file download from unleashed
add_new_unl(unl_download)  ## adds new direct purchases to raw database
clean_unleashed()          ## clean raw direct purchases and add to clean directdb

In [4]:
# UPLOAD AND CLEAN DOT

dot_download = r"C:\Users\mikej\Downloads\SHOP Supplier Reporting - Invoice Details Excel.xlsx"  ### file download from dot
add_new_dot(dot_download)  ## adds new indirect purchases to raw database
clean_dot()                ## clean raw indirect purchases and add to clean indirect db

In [5]:
# COMBINE clean_direct & clean_indirect POSTGRES TABLES TO GET NEW TABLE level_2 (or tRUE Sales)

get_level_2()              ## combine clean direct/indirect and populate level_2 db  --  dot invisible

In [ ]:
## copy postgres tables into CSV's for local use below

In [6]:
# create level_1.csv 
level_1 = pd.read_sql("SELECT * FROM unleashed_clean WHERE completed_date > '2022-12-31';", con = engine)

level_1.completed_date = pd.to_datetime(level_1.completed_date)
level_1['usd'] = level_1['sub_total']*.75

In [7]:
### create level_2.csv
level_2 = pd.read_sql("SELECT * FROM level_2 WHERE date > '2021-12-31';", con = engine)

level_2 = level_2[level_2.market_segment != 'Samples']
level_2.date = pd.to_datetime(level_2.date,format="%Y-%m-%d")

In [8]:
print("Latest Dates")
print(f"Direct: {level_1.completed_date.max()}")
print(f"tRUE:   {level_2.date.max()}")

Latest Dates
Direct: 2024-03-29 00:00:00
tRUE:   2024-07-05 00:00:00


In [9]:
# send csv's to data folder

level_1.to_csv(r"C:/Users/mikej/Desktop/cpg-sales/data/level_1.csv", index=False)
level_2.to_csv(r"C:/Users/mikej/Desktop/cpg-sales/data/level_2.csv", index=False)

In [10]:
# level_1_tableau

l1_tableau = level_1.drop(columns=['month','year'])
l1_tableau.columns = ['Customer Name', 'Invoice Date','Item Full Description', 'Quantity','Sub Total', 'Dollars', 'Table','Market Segment', 'Parent Customer']

l1_tableau['Table'] = 'Unleashed'

In [11]:
# level_2_tableau

# fix columns to mimic tableau download data output
l2_tableau = level_2.drop(columns=['qty','cad','month','year'])
l2_tableau.columns = ['Invoice Date', 'Sale Origin', 'Market Segment', 'Parent Customer', 'Customer', 'Item Full Description', 'Dollars', ]

# add vistar retail y/n and Sales Goal columns
l2_tableau['Vistar Retail'] = np.where(l2_tableau['Parent Customer'] == 'Vistar Retail', str('Yes'), str('No'))
l2_tableau['Sales Goal'] = l2_tableau.Dollars*1.5

In [12]:
# send tableau-ready tables to data folder

l2_tableau.to_csv(r"C:\Users\mikej\Desktop\cpg-sales\data\level_2_tableau.csv", index=False)
l1_tableau.to_csv(r"C:\Users\mikej\Desktop\cpg-sales\data\level_1_tableau.csv", index=False)

In [13]:
### check Direct Sales (level_1) daily

l1_bar_df = level_1.groupby('completed_date')['usd'].sum().reset_index().set_index('completed_date')
l1_bar_df =round(l1_bar_df[l1_bar_df.index>'2024-02-29']).sort_index()

level_1_bar = px.bar(l1_bar_df,
                     y='usd',
                     labels={'usd':'',
                             'completed_date':''},
                     height=325,
                     text_auto=",.2s").update_traces(textposition='outside')

level_1_bar.show()

In [15]:
### check tRUE Sales (level_2) daily

px.bar(level_2[level_2.date>'2023-12-31'].set_index('date').groupby(pd.Grouper(freq='d'))['usd'].sum(),
        y='usd',
        labels={'usd':'$USD','date':''},
        text_auto=",.2s",
        title='Daily tRUE Sales in USD',
        height=400).update_traces(textposition='outside')

In [17]:
level_2

,date,sale_origin,market_segment,parent_customer,customer,item,qty,usd,cad,month,year
0,2024-07-05,dot,Alternate Retail,Tropical,TROPICAL FOODS--NV,CAFFEINATED CHOCOLATE BITES SINGLES CARAMEL,1.0,141.00,187.53,July,2024
1,2024-07-05,dot,Vending,Vistar,VISTAR-HOUSTON,CAFFEINATED CHOCOLATE BITES SINGLES DARK,9.0,1269.00,1687.77,July,2024
2,2024-07-05,dot,Alternate Retail,Tropical,TROPICAL FOODS--TX,CAFFEINATED CHOCOLATE BITES SINGLES,9.0,1269.00,1687.77,July,2024
3,2024-07-05,dot,Vending,Vistar,VISTAR-MID-ATLANTIC,CAFFEINATED CHOCOLATE BITES SINGLES DARK,1.0,141.00,187.53,July,2024
4,2024-07-05,dot,Alternate Retail,Tropical,TROPICAL FOODS--TX,AWAKE CHOCOLATE AWAKE CAFF MILK CHOC- 6X12PK M...,3.0,256.62,341.30,July,2024
...,...,...,...,...,...,...,...,...,...,...,...
108531,2022-01-03,unl,Online,Shopify,Shopify Customer - mello USA,Mello Chocolate - Milk Pouch 102g / 3.6oz,1.0,0.00,0.00,January,2022
108532,2022-01-03,dot,Broadline Distributor,US Foods,USF/MEMPHIS-USF DIRECT,AWAKE CHOCOLATE AWAKE CAFF MILK CHOC- 6X12PK M...,2.0,156.96,208.76,January,2022
108533,2022-01-03,dot,Broadline Distributor,US Foods,USF/MEMPHIS-USF DIRECT,AWAKE CHOCOLATE AWAKE CHOC CARAMEL- 6X12PK MASTER,2.0,156.96,208.76,January,2022
108534,2022-01-03,unl,Online,Shopify,Shopify Customer - mello USA,Mello Chocolate - Milk Pouch 102g / 3.6oz,1.0,0.00,0.00,January,2022


In [29]:
### check tRUE Sales (level_2) all years

px.bar(level_2[level_2.sale_origin=='dot'].set_index('date').groupby(pd.Grouper(freq='M'))['usd'].sum(),
        y='usd',
        labels={'usd':'$USD','date':''},
        text_auto=",.2s",
        title='Monthly tRUE Sales in USD',
        height=400).update_traces(textposition='outside')

In [22]:
for market in level_2.market_segment.unique():
    
#     fig = px.bar(level_2[(level_2.market_segment==market) & (level_2.year==2024)],
    fig = px.scatter(level_2[(level_2.market_segment==market) & (level_2.year==2024)],
                     x='date',
                     y='usd',
                     title=market,
                     color = 'market_segment',
                     color_discrete_map = market_segment_dict,
                     height = 300,
                     width = 1000,
                     trendline = 'rolling', trendline_options=dict(window=21))
    fig.show()

In [23]:
level_2.columns

Index(['date', 'sale_origin', 'market_segment', 'parent_customer', 'customer',
       'item', 'qty', 'usd', 'cad', 'month', 'year'],
      dtype='object')